In [ ]:
#Detect transfer regions and clonal regions
#moving window =300bp
#heterozygosity threshold = 0.025

from itertools import combinations
from Bio import SeqIO
import os
import numpy as np

def count_zero_snp_windows(L, w, snp_positions):
    total_windows = L - w + 1
    affected = np.zeros(total_windows, dtype=bool)
    
    for p in snp_positions:
        start = max(0, p - w)
        end = min(total_windows, p)
        affected[start:end] = True  # Vectorized marking
    
    return np.count_nonzero(~affected)

def calculate_moving_window_heterozygosity(sequence1, sequence2, window_size):
    L = len(sequence1)
    diff_array = (sequence1 != sequence2).astype(int)
    cumsum_diff = np.cumsum(np.insert(diff_array, 0, 0))
    heterozygosity = (cumsum_diff[window_size:] - cumsum_diff[:-window_size]) / window_size
    return heterozygosity

def identify_recombined_regions(heterozygosity_values, window_size):
    recombined_regions = []
    in_region = False
    region_start = None
    snp_count = 0
    total_length = len(heterozygosity_values) * window_size
    recombined_starts=[]
    recombined_ends=[]
    
    for i, h in enumerate(heterozygosity_values):
        if h > 0:  # Start of a non-zero heterozygosity region
            if not in_region:
                region_start = i
                in_region = True
            snp_count += h
        else:  # End of a recombined region
            if in_region:
                region_end = i
                length = (region_end - region_start)
                avg_heterozygosity = snp_count / length
                if avg_heterozygosity > 0.025:
                    recombined_regions.append((length, avg_heterozygosity))
                    recombined_starts.append(region_start)
                    recombined_ends.append(region_end)
                in_region = False
                snp_count = 0
    if in_region:  # If still in a recombined region at the end
        region_end = len(heterozygosity_values)
        length = region_end - region_start
        avg_heterozygosity = snp_count / length
        if avg_heterozygosity > 0.025:
            recombined_regions.append((length, avg_heterozygosity))
            recombined_starts.append(region_start)
            recombined_ends.append(region_end)
    # Compute SNP rate outside recombined regions
    #snps_outside = total_snps - sum(r[0] * r[1] for r in recombined_regions)
    #length_outside = total_length - sum(r[0] for r in recombined_regions)
    #snp_rate_outside = snps_outside
    
    return recombined_regions,recombined_starts,recombined_ends

def process_alignment(file_path, output_dir, w=300):
    sequences = {record.id: np.array(list(str(record.seq))) for record in SeqIO.parse(file_path, "fasta")}
    seq_ids = list(sequences.keys())
    
    votu_name = os.path.splitext(os.path.basename(file_path))[0]  # Extract vOTU name
    
    count_over_80 = 0
    clonal_v_recombined = []
    votu_dir = os.path.join(output_dir, votu_name)
    os.makedirs(votu_dir, exist_ok=True)
    recombination_file = os.path.join(votu_dir, f"{votu_name}_recombined_regions.txt")
    with open(recombination_file, "w") as r_f:
        r_f.write("seq1\tseq2\tlength_bp\taverage_heterozygosity_in_recombined_region\tfraction_identical\n")
        for seq1, seq2 in combinations(seq_ids, 2):
            s1, s2 = sequences[seq1], sequences[seq2]
            
            # Remove gap sites from both sequences
            valid_sites = (s1 != '-') & (s2 != '-')
            s1, s2 = s1[valid_sites], s2[valid_sites]
            
            differing_sites = np.where(s1 != s2)[0]  # Vectorized SNP detection
            total_snps=len(differing_sites)
            identical_windows = count_zero_snp_windows(len(s1), 300, differing_sites)
            fraction_identical = identical_windows / (len(s1) - w + 1)
            
            if fraction_identical > 0.6:
                count_over_80 += 1
                heterozygosity_values = calculate_moving_window_heterozygosity(s1, s2, w)
                recombined_regions, recombined_starts,recombined_ends = identify_recombined_regions(heterozygosity_values, w)
                excluded_snps = np.sum([np.sum((differing_sites >= start) & (differing_sites < end)) 
                            for start, end in zip(recombined_starts, recombined_ends)])
                snps_outside = total_snps - excluded_snps
                snp_rate_outside=snps_outside
                num_recombined = len(recombined_regions)
                clonal_v_recombined.append([seq1, seq2, num_recombined, snp_rate_outside, fraction_identical])
                for length, avg_heterozygosity in recombined_regions:
                    r_f.write(f"{seq1}\t{seq2}\t{length}\t{avg_heterozygosity}\t{fraction_identical}\n")

    recombination_file2 = os.path.join(votu_dir, f"{votu_name}_recombined_regions_clonal_v_recombined_frac_pairs_over_80_{count_over_80}.txt")

    with open(recombination_file2, "w") as r_f:
        r_f.write("seq1\tseq2\tnum_recombined\tclonal_snps\tfraction_identical\n")
        for row in clonal_v_recombined:
            seq1, seq2, num_recombined, sro, fi = row
            r_f.write(f"{seq1}\t{seq2}\t{num_recombined}\t{sro}\t{fi}\n")

def process_all_files(input_dir, output_dir, w=300):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".fna"):  # Process only fasta files
            file_path = os.path.join(input_dir, file_name)
            process_alignment(file_path, output_dir, w)

# Example usage:
input_dir = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/large_vOTU_core_genome_alignments"
output_dir = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/Fraction_identical_over_80_hgt_events_fi60"
process_all_files(input_dir, output_dir)



In [ ]:
#plot clonal SNPs vs recombined regions
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

# Folder containing vOTU directories
base_folder = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/Fraction_identical_over_80_hgt_events_fi60"
mean_length_file = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/votu_mean_lengths.txt"

# Identify vOTU folders and extract their sizes
votu_info = []
pattern = re.compile(r"pangraph_vOTU-(\d+)_n(\d+)_")

for folder in os.listdir(base_folder):
    match = pattern.search(folder)
    if match:
        votu_id = f"vOTU-{match.group(1)}"
        size = int(match.group(2))  # Extract n value
        votu_info.append((votu_id, size, os.path.join(base_folder, folder)))

# Select the 100 largest vOTUs
top_votus = sorted(votu_info, key=lambda x: x[1], reverse=True)[:60]

# Load mean genome lengths from file
mean_lengths = {}
with open(mean_length_file, "r") as f:
    for line in f:
        votu, mean_length = line.strip().split("\t")
        mean_lengths[votu] = float(mean_length)

# Collect valid input file paths (with at least 20 recombination events)
file_paths = {}
for votu_id, size, votu_folder in top_votus:
    
    matching_files = [file for file in os.listdir(votu_folder) if "frac_pairs" in file]
    input_file = votu_folder+'/'+matching_files[0]
    
    if os.path.exists(input_file):
        # Check if the file has more than 10 recombination events (rows)
        df = pd.read_csv(input_file, sep='\t')
        if len(df) > 10:
            mean_length = mean_lengths.get(votu_id, "N/A")
            label = f"{votu_id} (n={size})\n Genome Length={mean_length:.2f})" if mean_length != "N/A" else f"{votu_id} (n={size})"
            file_paths[label] = input_file

# Load and combine data
data = []
for label, path in file_paths.items():
    df = pd.read_csv(path, sep='\t')
    df["Dataset"] = label  # Label with vOTU ID, size, and mean genome length
    df = df[(df["clonal_snps"] != 0) & (df["num_recombined"] != 0)]
    df["ratio"] = df["clonal_snps"] / df["num_recombined"]
    data.append(df)

# Concatenate all datasets
df_all = pd.concat(data, ignore_index=True)

# Define bins
ratio_bins = np.append(np.logspace(-3, 3, num=15), np.inf)

# Assign bin labels
df_all["ratio_bin"] = pd.cut(df_all["ratio"], bins=ratio_bins)

# Set up figure with two subplots
fig, axes = plt.subplots(nrows=1, sharex=True, figsize=(20, 8))

# Violin plot for heterozygosity with transparency
sns.boxplot(x="Dataset", y="ratio", data=df_all, ax=axes, boxprops=dict(alpha=0.2),
            flierprops=dict(marker='o', color='w', markersize=0))
axes.set_xlabel("vOTU")
axes.set_ylabel("Clonal SNPS/Transfers")
axes.set_yscale('log')
axes.set_ylim(0.001, 1000)
axes.tick_params(axis='x', rotation=90)

# Scatter plot for data points below the box
# Add some Gaussian noise to spread the points slightly
noise_std = 3
df_all['y_noise'] = df_all['ratio'] + np.random.normal(0, noise_std, len(df_all))

# Define a threshold for outliers (1.5 * IQR rule for boxplot outliers)
Q1 = df_all['ratio'].quantile(0.25)
Q3 = df_all['ratio'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold_upper = Q3 + 1.5 * IQR
outlier_threshold_lower = Q1 - 1.5 * IQR

# Filter out the outliers
df_all_no_outliers = df_all[(df_all['ratio'] <= outlier_threshold_upper) & (df_all['ratio'] >= outlier_threshold_lower)]

# Scatter the data points
sns.stripplot(x="Dataset", y="y_noise", data=df_all, ax=axes, color='blue', jitter=True, alpha=0.5,s=2)

plt.tight_layout()
plt.show()


In [ ]:
#plot length and heterozygosity of transfers

import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Folder containing vOTU directories
base_folder = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/Fraction_identical_over_80_hgt_events_fi60"
mean_length_file = "/Users/jferrare/Documents/Good Lab Work/Gut Phage/votu_mean_lengths.txt"

# Identify vOTU folders and extract their sizes
votu_info = []
pattern = re.compile(r"pangraph_vOTU-(\d+)_n(\d+)_")

for folder in os.listdir(base_folder):
    match = pattern.search(folder)
    if match:
        votu_id = f"vOTU-{match.group(1)}"
        size = int(match.group(2))  # Extract n value
        votu_info.append((votu_id, size, os.path.join(base_folder, folder)))

# Select the 100 largest vOTUs
top_votus = sorted(votu_info, key=lambda x: x[1], reverse=True)[:40]

# Load mean genome lengths from file
mean_lengths = {}
with open(mean_length_file, "r") as f:
    for line in f:
        votu, mean_length = line.strip().split("\t")
        mean_lengths[votu] = float(mean_length)

# Collect valid input file paths (with at least 20 recombination events)
file_paths = {}
for votu_id, size, votu_folder in top_votus:
    input_file = os.path.join(votu_folder, f"{os.path.basename(votu_folder)}_recombined_regions.txt")
    
    if os.path.exists(input_file):
        # Check if the file has more than 20 recombination events (rows)
        df = pd.read_csv(input_file, sep='\t')
        if len(df) > 50:
            mean_length = mean_lengths.get(votu_id, "N/A")
            label = f"{votu_id} (n={size})\n Genome Length={mean_length:.2f})" if mean_length != "N/A" else f"{votu_id} (n={size})"
            file_paths[label] = input_file

# Load and combine data
data = []
for label, path in file_paths.items():
    df = pd.read_csv(path, sep='\t')
    df["Dataset"] = label  # Label with vOTU ID, size, and mean genome length
    data.append(df)

# Concatenate all datasets
df_all = pd.concat(data, ignore_index=True)

# Define bins
hetero_bins = np.append(np.linspace(0.02, 0.1, num=15), np.inf)
length_bins = np.logspace(np.log10(50), np.log10(10000), num=15)

# Assign bin labels
df_all["hetero_bin"] = pd.cut(df_all["average_heterozygosity_in_recombined_region"], bins=hetero_bins)
df_all["length_bin"] = pd.cut(df_all["length_bp"], bins=length_bins)

# Set up figure with two subplots
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(20, 8))

# Violin plot for heterozygosity
sns.violinplot(x="Dataset", y="average_heterozygosity_in_recombined_region", data=df_all, ax=axes[0], scale="width", cut=0)
axes[0].set_xlabel("vOTU (size, mean genome length)")
axes[0].set_ylabel("Heterozygosity in \n recombined regions")
axes[0].set_ylim(0.02, 0.1)
axes[0].tick_params(axis='x', rotation=90)

# Violin plot for lengths (log-scaled)
sns.violinplot(x="Dataset", y="length_bp", data=df_all, ax=axes[1], scale="width", cut=0)
axes[1].set_yscale("log")
axes[1].set_ylim(50, 10000)
axes[1].set_ylabel("Length of \n recombined regions")
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()
